# Data Preparation
In this notebook we will prepare our data for our search function to use. Currently we have data stored in three different csv file.

. College_data.csv

. ratings.csv

. fees.csv

. tag.csv


It can be computationally expensive to produce analysis results from multiple data-sources for incomming stream of requests. So we will prepare our data and save it in an easily searchable structure

In [45]:
# Import the needed modules...
import numpy as np
import pandas as pd
from collections import defaultdict
from os import getcwd

# Define Paths to data file.

In [46]:
PATH_COLLEGE   = f"{getcwd()}/dataStore/college.csv"
PATH_RATINGS = f"{getcwd()}/dataStore/ratings.csv"
PATH_FEES   = f"{getcwd()}/dataStore/fees.csv"
PATH_TAG = f"{getcwd()}/dataStore/tag.csv"

# Data Engineering

# Get Data in Dataframes.


 # Convert data to a single Dictionary.

In [47]:
"""
    Read data from College.csv
"""
df_college  = pd.read_csv(PATH_COLLEGE)
college_table_columns = df_college.columns.tolist()
print(f"COLUMNS : {college_table_columns}")

COLUMNS : ['college_id', 'College_Name', 'State', 'Stream']


In [48]:
"""
    Read data from ratings.csv
"""
df_ratings         = pd.read_csv(PATH_RATINGS)
path_table_columns = df_ratings.columns.tolist()
print(f"COLUMNS : {path_table_columns}")

COLUMNS : ['userId', 'college_id', 'Rating', 'Academic', 'Accommodation', 'Faculty', 'Infrastructure', 'Placement', 'Social_Life', 'timestamp']


In [49]:
"""
    Read data from fees.csv
"""
df_fees         = pd.read_csv(PATH_FEES)
fees_table_columns = df_fees.columns.tolist()
print(f"COLUMNS : {fees_table_columns}")

COLUMNS : ['college_id', 'UG_fee', 'PG_fee']


In [50]:
"""
    Read data from tag.csv
"""
df_tag         = pd.read_csv(PATH_TAG)
tag_table_columns = df_tag.columns.tolist()
print(f"COLUMNS : {tag_table_columns}")

COLUMNS : ['userId', 'college_id', 'tag', 'timestamp']


# 
college_id is a common column in all four tables so we will use it as a primary search-key


userId is a common key across two tables, so we will use it as a sort key...


A user will always search a college by its name so we will create a Global secondary index to be able to perform search our datastore.
it will obviously take some extra space but almost negligible as compared to the size of the original data.
In addition, It will make our searching faster and efficient so it's a good deal.

In [51]:
print(f"It is {pd.Series(df_college['college_id']).is_unique}  that the column 'id' has unique values for all entries in college dataframe.")
print(f"It is {pd.Series(df_fees['college_id']).is_unique}  that the column 'id' has unique values for all entries in fees dataframe.")
print(f"It is {pd.Series(df_ratings['userId']).is_unique} that the column 'userId'  has unique values for all entries in ratings dataframe.")
print(f"It is {pd.Series(df_tag['userId']).is_unique} that the column 'userId'  has unique values for all entries in tags dataframe.")

# Sort college dataframe on the basis of movieId as movieId is unique for all entries...
df_college_sorted = df_college.sort_values(by=['college_id'])

# Sort fees dataframe on the basis of movieId as movieId is unique for all entries...
df_fees_sorted  = df_fees.sort_values(by=['college_id'])


It is True  that the column 'id' has unique values for all entries in college dataframe.
It is True  that the column 'id' has unique values for all entries in fees dataframe.
It is False that the column 'userId'  has unique values for all entries in ratings dataframe.
It is False that the column 'userId'  has unique values for all entries in tags dataframe.


In [52]:
# from college dataframe...
collegeIds = df_college_sorted["college_id"].tolist()
College_Names = df_college_sorted["College_Name"].tolist()
States = df_college_sorted["State"].tolist()
Streams = df_college_sorted["Stream"].tolist()
# from fees dataframe...
UG_fees  = df_fees_sorted["UG_fee"].tolist()
PG_fees  = df_fees_sorted["PG_fee"].tolist()

In [53]:
collegeDict             = {}
global_secondaryIndex = {}
for idx, college_id in enumerate(collegeIds):
    collegeDict[college_id] = {
        "Streams" : Stream[idx],
        "States"  : State[idx],
        "fees" : {
            "UG_fees" : UG_fee[idx], 
            "PG_fees" : PG_fee[idx]
    }
    }
    
    global_secondaryIndex[College_Names[idx]] = college_id

In [54]:
# delete veriables which are no longer in use while holding large amount of data.
del College_Names
del States
del Streams
del UG_fees
del PG_fees

### Add all user ratings for individual college.


The goal is to group all ratings of a college together, so that we will be able to retrieve user ratings of a particular college.

Now, this one is a bit tricky as there is no column in the ratings dataframe which offers unique values.

So will have to perform grouping.

We will use college_id column as it is a common column in all of our data sources and it will make it easy to add the same data in our new college dataset.

The procedure defined below may be computationally gross but should be good enough for a single time execution...

In [55]:
# convert all columns of ratings table into individual lists...
userIds       = df_ratings["userId"].tolist()
collegeIds      = df_ratings["college_id"].tolist()
user_ratings  = df_ratings["Rating"].tolist()
timestamps    = df_ratings["timestamp"].tolist()

ratings = {}

for idx, mid in enumerate(collegeIds):
    # Do the college_d previously exist?
    try   : _ = ratings[mid]
    # If not, Add it in the record...
    except: 
        ratings[mid]   = [
            {
                "userId"     : userIds[idx],
                "rating"     : user_ratings[idx],
                "time_stamp" : timestamps[idx]
            }
        ]
    
    try   : _ = ratings[mid][userIds[idx]]
    except: ratings[mid].append(
            {
                "userId"     : userIds[idx],
                "rating"     : user_ratings[idx],
                "time_stamp" : timestamps[idx]
            }
        )

# Finally, add the data in the collegeDict...
for mid, _ in collegeDict.items():
    try   : collegeDict[mid]["user_rating"] = ratings[mid][1:]
    except: 
        try   : collegeDict[mid]["user_rating"] = [] # If college_id exists in the college dict...
        except: pass # If the college_id doesn't exist in our record...

###  Add all user given tags for individual college.

The goal is to group all tags given to a college togather, so that we will be able to retrieve tags of a particular college.

This one is also tricky as there is no column in the tags dataframe which offers unique values.

So will have to perform grouping.

We will use college_id column as it is a common column in all of our data sources and it will make it easy to add the same data in our new college dataset.

The procedure defined below may also be computationally gross but should be good enough for a single time execution...

In [56]:
# convert all columns of ratings table into individual lists...
userIds    = df_tag["userId"].tolist()
collegeIds   = df_tag["college_id"].tolist()
user_tags   = df_tag["tag"].tolist()
timestamps = df_tag["timestamp"].tolist()

tag = {}
for idx, mid in enumerate(collegeIds):
    # Do the college_id previously exist?
    try   : _ = tag[mid]
    # If not, Add it in the record...
    except: tag[mid] = [
        {
            "userId"     : userIds[idx],
            "rating"     : user_tags[idx],
            "time_stamp" : timestamps[idx]
        }
    ]
    
    try   : _ = ratings[mid][userIds[idx]]
    except: tag[mid].append(
            {
                "userId"     : userIds[idx],
                "rating"     : user_tags[idx],
                "time_stamp" : timestamps[idx]
            }
        )

# Finally, add the data in the collegeDict...
for mid, _ in collegeDict.items():
    try   : collegeDict[mid]["tag"] = tag[mid][1:]
    except: 
        try   : collegeDict[mid]["tag"] = [] # If college_id exists in the college dict...
        except: del global_secondaryIndex[mid]

In [57]:
import json
print("[INFO] Writing college Data into the disk...")
with open('dataStore/dataFinal.json', 'w') as fp:
    json.dump(collegeDict, fp, sort_keys=True, indent=4)
print("[INFO] Writing Global Secondary Index Data into the disk...")
with open('dataStore/dataFinal_GIS.json', 'w') as fp:
    json.dump(global_secondaryIndex, fp, sort_keys=True, indent=4)

[INFO] Writing college Data into the disk...
[INFO] Writing Global Secondary Index Data into the disk...


##### At this point, our database is ready and it can handel high inflow of requests.